In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/autism-screening-for-toddlers/Toddler Autism dataset July 2018.csv')

In [ ]:
df.columns


In [ ]:
df.drop(['Case_No','Who completed the test'], axis=1, inplace=True)


In [ ]:
df.columns

In [ ]:
import seaborn as sns
sns.countplot(df['Class/ASD Traits '])

The data is a little bias? Majority are classfied as having ASD

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10,10))
ax = sns.heatmap(df.corr(),annot=True,cmap='coolwarm',vmin=0, vmax=1)

A10 and Age_Mons are not really correlated with the score

In [ ]:
sns.scatterplot(x='Age_Mons',y='Qchat-10-Score',data=df)

okay, it looks like it doesn't matter what your age is, the chance of the getting high/low score won't be affected.

In [ ]:
sns.scatterplot(x='A10',y='Qchat-10-Score',data=df)

A10 doesn't seem like a good indicator as well. 

I want to see what are the ethicity distribution among the survey responders. How common is that among the gender and ethicity? 

In [ ]:
sns.countplot(df['Ethnicity'])
plt.xticks(rotation=90)

In [ ]:
# df.columns
# df_new = df[df['Class/ASD Traits ']=='Yes']
sns.countplot(x='Ethnicity',hue='Sex',data=df)
plt.xticks(rotation=90)

#we can see that more survey respondants are males than females for each ethnicity

In [ ]:
df_plot = df.groupby(['Sex', 'Ethnicity']).size().reset_index().pivot(columns='Sex', index='Ethnicity', values=0)
# df_plot
df_plot.plot(kind='bar', stacked=True)

In [ ]:
x, y, hue = 'Ethnicity', 'prop', 'Sex'
prop_df = (df[x]
            .groupby(df[hue])
            .value_counts(normalize=True)
            .rename(y)
            .reset_index())
sns.barplot(x=x,y=y,hue=hue,data=prop_df)
plt.xticks(rotation=90)

In [ ]:
# sns.countplot(df['Sex'])
df = df[df['Class/ASD Traits ']=='Yes']
sns.countplot(df['Sex'])
#we can see more males are are categorized as ASD thru the app

Q: how many of the males/males are ASD? What is the proportion? 

In [ ]:
df_yes = df[df['Class/ASD Traits ']=='Yes']

m_prop = df_yes.Sex.value_counts()['m']/df.Sex.value_counts()['m']
f_prop = df_yes.Sex.value_counts()['f']/df.Sex.value_counts()['f']
m_prop
f_prop

This dataset is extremely bias? on the classification. It might causes problems with unmet data. 

In [ ]:
df.info()

In [ ]:
from sklearn.preprocessing import OneHotEncoder
cat_features = df.select_dtypes(include='object')
num_features = df.select_dtypes(exclude='object')
df = pd.get_dummies(df, drop_first=True)
df

In [ ]:
from sklearn.model_selection import train_test_split
X = df.drop(['Class/ASD Traits _Yes'], axis=1)
y = df['Class/ASD Traits _Yes']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [ ]:
#logistic method
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
preds = logreg.predict(X_test)
logreg.score(X_train, y_train)

In [ ]:
#classification report
#confusion matrix
from sklearn.metrics import classification_report, confusion_matrix
classification_report(y_test, preds)


In [ ]:
confusion_matrix(y_test,preds)

Well, it is suprising that logistic regression works perfectly fine? There's something wrong with this model or dataset. 

In [ ]:
#Let's try another model
#KNN
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)
pred = knn.predict(X_test)
classification_report(y_test, pred)


In [ ]:
confusion_matrix(y_test, pred)

In [ ]:
#to get the optimum n_neighbors
#this code is taken from a Udemy course "Python for Data Science and Machine Learning" taught by Jose Portilla
import matplotlib.pyplot as plt
error_rate = []
for i in range(1,40):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, y_train)
    pred = knn.predict(X_test)
    error_rate.append(np.mean(y_test != pred))
plt.figure(figsize=(10,10))
plt.plot(range(1,40), error_rate, color='blue',linestyle='dashed',marker='o',markerfacecolor='red')

the error rate dropped when it is around 27. We will set the n_neighbors to 27, and let's see what will happen. 

In [ ]:
knn = KNeighborsClassifier(n_neighbors=27)
knn.fit(X_train, y_train)
pred = knn.predict(X_test)
classification_report(y_test, pred)

In [ ]:
confusion_matrix(y_test, pred)

ok. much better result! Next step is Decision Tree and random forest. 

In [ ]:
#decision tree
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier()
dtree.fit(X_train, y_train)
pred = dtree.predict(X_test)
classification_report(y_test,pred)

In [ ]:
confusion_matrix(y_test,pred)

Did I do something wrong? Why is the score so good?

In [ ]:
#let's try random forest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
pred = rf.predict(X_test)
classification_report(y_test,pred)

In [ ]:
confusion_matrix(y_test,pred)

In [ ]:
#one last model, which I will use SVM

In [ ]:
from sklearn.svm import SVC
model = SVC()
model.fit(X_train, y_train)
pred = model.predict(X_test)
classification_report(y_test,pred)

In [ ]:
confusion_matrix(y_test,pred)

Conclusion: Logistic, tree, random forest, KNN, and SVM predicted well. 

I will end this notebook here. This is my first notebook, so please let me know if there's anything I can improve. I look forward to any feedback. 